In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cmcrameri.cm as cmc
from sklearn.metrics import mean_squared_error


In [ ]:
# Create a discrete colormap with 5 colors (one for ground truth, and one for each network architecture)
arch_names = ['g_true', 'FF', 'PI-FF', 'DGM', 'PI-DGM']
batlow_colors = cmc.batlow(np.linspace(0, 1, len(arch_names)))

# Create a custom palette dictionary mapping each class to a color
custom_palette = {arch_name: batlow_colors[i] for i, arch_name in enumerate(arch_names)}

In [ ]:
# Plate (2D)

alpha = 55
length = 50 # length of the plate, 50 mm
time = 8 # total time, 4 s
x_1_nodes = 30 # number of nodes - 1
x_2_nodes = 20 # made this different so that I could tell which axis is which

dx_1 = length / x_1_nodes # distance between nodes
dx_2 = length / x_2_nodes # distance between nodes
# time step, must be less or equal to than min of dx^2 / (4 * alpha) and dy^2 / (4 * alpha)
dt = np.min([0.25 * dx_1**2 / alpha, 0.25 * dx_1**2 / alpha]) 

x_1_train = np.linspace(0, length, x_1_nodes)
x_2_train = np.linspace(0, length, x_2_nodes)
t_all = np.linspace(0, time, int(time / dt))
train_low = len(t_all) // 8 # discard 1st 1/8th of time
train_high = 2 * train_low 
t_train = t_all[train_low:train_high] # train on 2nd 1/8th of time
t_test = t_all[train_high:]

u = np.zeros((x_1_nodes, x_2_nodes)) + 20 # middle of the plate is 20 degrees
# for i in range(len(u[0, :])): # bottom ranges from 20 to 78 degrees linearly
#     u[0, i] = 20 + 3*i
u[0, :] = 100 # bottom side of the plate is 100 degrees
# u[-1, :] = 100 # top side of the plate is 100 degrees
# u[:, 0] = 100 # left side of the plate is 100 degrees
# u[:, -1] = 100 # right side of the plate is 100 degrees

heat_data = np.zeros((int(time / dt), x_1_nodes, x_2_nodes))

for counter in range(heat_data.shape[0]):
    w = u.copy()

    for i in range(1, x_1_nodes - 1):
        for j in range(1, x_2_nodes - 1):
            dd_ux_1 = (w[i - 1, j] - 2 * w[i, j] + w[i + 1, j]) / dx_1**2
            dd_ux_2 = (w[i, j - 1] - 2 * w[i, j] + w[i, j + 1]) / dx_2**2

            u[i, j] = dt * alpha * (dd_ux_1 + dd_ux_2) + w[i, j]

    heat_data[counter, :, :] = u.copy()

    # print(f't: {counter * dt:.2f} s, Ave temp: {np.mean(u):.2f} C')

print(heat_data.shape)

In [ ]:
def plot_heat_map(
        title,
        samp_time,
        x_1,
        x_2,
        heat
):
    fig = plt.figure()
    plt.title(f'{title}\nt: {samp_time:.2f} s')
    pcm = plt.pcolormesh(x_2, x_1, heat, cmap='cmc.batlow', vmin=0, vmax=100)
    cbar = plt.colorbar(pcm)
    cbar.set_label('Temperature (ºC)')
    plt.xlabel('$x_{2}$ mm')
    plt.ylabel('$x_{1}$ mm')
    plt.show()


In [ ]:
samp_time = len(t_train) // 2 # plot the time in the middle of training

plot_heat_map('Ground Truth', (len(t_all))*dt, x_1_train, x_2_train, heat_data[-1, :, :])


In [ ]:
import os
import sys

parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)

from deep_learning import deep_network_core as core, utils
import torch
import torch.nn as nn
from torch.autograd import grad as autograd

In [ ]:
# Create meshgrids for train and test  
Tr, X_1r, X_2r = np.meshgrid(t_train, x_1_train, x_2_train)

inpt = np.column_stack((
    Tr.transpose(1, 0, 2).ravel(), # Transpose needed to get in t, x, y order
    X_1r.transpose(1, 0, 2).ravel(), 
    X_2r.transpose(1, 0, 2).ravel(),
    ))
oupt = heat_data[train_low:train_high, :, :].ravel()

Tr_test, X_1r_test, X_2r_test = np.meshgrid(t_test, x_1_train, x_2_train)

inpt_test = np.column_stack((
    Tr_test.transpose(1, 0, 2).ravel(), # Transpose needed to get in t, x, y order
    X_1r_test.transpose(1, 0, 2).ravel(), 
    X_2r_test.transpose(1, 0, 2).ravel(),
    ))


In [ ]:
class MSE_Loss(core.LOSS):
    def __init__(self):
        self.loss = nn.MSELoss()
        
    def __call__(self, target, result, model):
        return self.loss(target, result)

class Huber_Loss(core.LOSS):
    def __init__(self):
        self.loss = nn.HuberLoss()
        
    def __call__(self, target, result, model):
        return self.loss(target, result)

class PHYSICS_Loss(core.LOSS):
    def __init__(self, alpha):
        self.alpha = alpha
    
    def __call__(self, target, result, model):
        x_1 = torch.empty((100, 1)).uniform_(0, length).requires_grad_(True)
        x_2 = torch.empty((100, 1)).uniform_(0, length).requires_grad_(True)
        t = torch.empty((100, 1)).uniform_(train_low*dt, time).requires_grad_(True)
        inp = torch.cat((t, x_1, x_2), axis=1)
        zs = model(inp)
        pde = utils.dy_dt(zs, t) - self.alpha * utils.laplacian_2d(zs, x_1, x_2) # dz/dt - (ddz/dx_1x_1 + ddz/dx_2x_2)
        return torch.mean(pde**2)

In [ ]:
network_nn = core.PINN(3, 1, 64, 3, [(1, MSE_Loss())])

print("Training FF No Physics")
network_nn.fit(inpt, oupt, lr=1e-3, epochs=1000)


In [ ]:
# Generate train and test data from the model for analysis and plotting
pred_nn_train = network_nn.predict(inpt).reshape((len(t_train), len(x_1_train), len(x_2_train)))
pred_nn_test = network_nn.predict(inpt_test).reshape((len(t_test), len(x_1_train), len(x_2_train)))

plot_heat_map('Feed Forward No Physics', (len(t_all))*dt, x_1_train, x_2_train, pred_nn_test[-1, :, :])


In [ ]:
mses_nn_train = []
for i, pred in enumerate(pred_nn_train):
    mses_nn_train.append(mean_squared_error(heat_data[train_low+i, :, :], pred))

mses_nn_test = []
for i, pred in enumerate(pred_nn_test):
    mses_nn_test.append(mean_squared_error(heat_data[train_high+i, :, :], pred))


In [ ]:
network_pinn = core.PINN(3, 1, 64, 3, [(1, MSE_Loss()), (1, PHYSICS_Loss(alpha))])

print("Training Feed Forward with Physics")
network_pinn.fit(inpt, oupt, lr=1e-3, epochs=1000)

In [ ]:
pred_pinn_train = network_pinn.predict(inpt).reshape((len(t_train), len(x_1_train), len(x_2_train)))
pred_pinn_test = network_pinn.predict(inpt_test).reshape((len(t_test), len(x_1_train), len(x_2_train)))

plot_heat_map('Feed Forward Physics', (len(t_all))*dt, x_1_train, x_2_train, pred_pinn_test[-1, :, :])


In [ ]:
mses_pinn_train = []
for i, pred in enumerate(pred_pinn_train):
    mses_pinn_train.append(mean_squared_error(heat_data[i+train_low, :, :], pred))

mses_pinn_test = []
for i, pred in enumerate(pred_pinn_test):
    mses_pinn_test.append(mean_squared_error(heat_data[train_high+i, :, :], pred))


In [ ]:
network_dgm = core.DGM(3, 1, 128, 4, [(1, MSE_Loss())])
print("Training DGM without Physics")
network_dgm.fit(inpt, oupt, lr=1e-2, epochs=200)

In [ ]:
pred_dgm_train = network_dgm.predict(inpt).reshape((len(t_train), len(x_1_train), len(x_2_train)))
pred_dgm_test = network_dgm.predict(inpt_test).reshape((len(t_test), len(x_1_train), len(x_2_train)))

plot_heat_map('DGM No Physics', (len(t_all))*dt, x_1_train, x_2_train, pred_dgm_test[-1, :, :])


In [ ]:

mses_dgm_train = []
for i, pred in enumerate(pred_dgm_train):
    mses_dgm_train.append(mean_squared_error(heat_data[train_low+i, :, :], pred))

mses_dgm_test = []
for i, pred in enumerate(pred_dgm_test):
    mses_dgm_test.append(mean_squared_error(heat_data[train_high+i, :, :], pred))


In [ ]:
network_pi_dgm = core.DGM(3, 1, 128, 4, [(1, MSE_Loss()), (1, PHYSICS_Loss(alpha))])
print("Training DGM with Physics")
network_pi_dgm.fit(inpt, oupt, lr=1e-2, epochs=200)

In [ ]:
pred_pi_dgm_train = network_pi_dgm.predict(inpt).reshape((len(t_train), len(x_1_train), len(x_2_train)))
pred_pi_dgm_test = network_pi_dgm.predict(inpt_test).reshape((len(t_test), len(x_1_train), len(x_2_train)))

plot_heat_map('DGM Physics', (len(t_all))*dt, x_1_train, x_2_train, pred_pi_dgm_test[-1, :, :])



In [ ]:
mses_pi_dgm_train = []
for i, pred in enumerate(pred_pi_dgm_train):
    mses_pi_dgm_train.append(mean_squared_error(heat_data[train_low+i, :, :], pred))

mses_pi_dgm_test = []
for i, pred in enumerate(pred_pi_dgm_test):
    mses_pi_dgm_test.append(mean_squared_error(heat_data[train_high+i, :, :], pred))


In [ ]:
plt.figure()
plt.semilogy(t_train, mses_nn_train, '-', c=batlow_colors[1])
plt.semilogy(t_train, mses_pinn_train, '-', c=batlow_colors[2])
plt.semilogy(t_train, mses_dgm_train, '-', c=batlow_colors[3])
plt.semilogy(t_train, mses_pi_dgm_train, '-', c=batlow_colors[4])
plt.semilogy(t_test, mses_nn_test, '--', c=batlow_colors[1])
plt.semilogy(t_test, mses_pinn_test, '--', c=batlow_colors[2])
plt.semilogy(t_test, mses_dgm_test, '--', c=batlow_colors[3])
plt.semilogy(t_test, mses_pi_dgm_test, '--', c=batlow_colors[4])
plt.grid(True)
plt.ylabel('Mean Squared Error')
plt.xlabel('Time (s)')
plt.title('Mean Squared Error for Each Time Step')
plt.legend(['FF No Physics', 'FF Physics', 'DGM No Physics', 'DGM Physics'])
plt.show()

In [ ]:
# np.savez(
#     'pred_values_alpha.npz', 
#     nn_train_data=pred_nn_train,
#     nn_test_data=pred_nn_test,
#     pinn_train_data=pred_pinn_train,
#     pinn_test_data=pred_pinn_test,
#     dgm_train_data=pred_dgm_train,
#     dgm_test_data=pred_dgm_test,
#     pi_dgm_train_data=pred_pi_dgm_train,
#     pi_dgm_test_data=pred_pi_dgm_test,
#     )

In [ ]:
# loaded = np.load('pred_values_alpha55_work.npz')
# pred_nn_train = loaded['nn_train_data']
# pred_nn_test = loaded['nn_test_data']
# pred_pinn_train = loaded['pinn_train_data']
# pred_pinn_test = loaded['pinn_test_data']
# pred_dgm_train = loaded['dgm_train_data']
# pred_dgm_test = loaded['dgm_test_data']
# pred_pi_dgm_train = loaded['pi_dgm_train_data']
# pred_pi_dgm_test = loaded['pi_dgm_test_data']
